In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
df_2 = pd.read_csv('szereg_czasowy.csv', sep=";")

df_2 = df_2.dropna(subset=['retail_price'])
df_2 = df_2.reset_index(drop=True)
df_2 = df_2.sort_values(by='year')
df_2 = df_2.replace(',', '.')

df_2

In [ ]:
average_values = df_2.iloc[:, 8:].apply(lambda x: x.mean(), axis=0)
average_values_list = average_values.tolist()

average_values_rounded = [round(x, 6) for x in average_values_list]

average_values_rounded_series = pd.Series(average_values_rounded, name='average_values_rounded')
average_values_rounded_series.name = 'average_values_rounded'
df_2['average_values_rounded'] = average_values_rounded_series

LEGO_basket = pd.DataFrame(average_values_rounded_series)

In [ ]:
yf.pdr_override()
other_instruments = pdr.get_data_yahoo(["GC=F", "^GSPC", "^GDAXI", "BTC-USD"], start="2021-01-01", end="2023-12-31")
other_instruments = other_instruments.resample('M').last()
other_instruments = other_instruments['Close']
other_instruments = other_instruments.reset_index()

#merge LEGO_basket and other_investments
all_instruments = pd.concat([other_instruments, LEGO_basket], axis=1)
all_instruments= all_instruments.rename(columns={'GC=F': 'Gold', '^GSPC': 'SP500', 'WIG20.WA': 'WIG20', '^GDAXI': 'DAX','average_values_rounded': 'LEGO'})

#Set 'Date' as index
all_instruments.set_index("Date", inplace=True)
all_instruments.plot(figsize=(5,8), subplots=True)

# Set the minimum value on the y-axis to 0 for all subplots
for ax in plt.gcf().get_axes():
    ax.set_ylim(bottom=0)

plt.show()

In [ ]:
all_instruments.plot(figsize=(5,8), subplots=True)

In [ ]:
all_instruments.tail()

In [ ]:
x = all_instruments.index.to_numpy()
y = all_instruments['LEGO'].to_numpy()

# Convert the datetime64 array to a Unix timestamp array
x_unix_timestamps = (x - np.datetime64('1970-01-01')) / np.timedelta64(1)

# Fit a polynomial of degree 1 (a straight line) to the data
fit = np.polyfit(x_unix_timestamps, y, 1)

# Generate the equation of the fit line
fit_line = np.poly1d(fit)

# Plot the original data
plt.plot(x, y, label='Koszyk LEGO')

# Plot the fit line
plt.plot(x, fit_line(x_unix_timestamps), 'r--', label='Linia trendu')

# Show the plot
plt.title('Badany koszyk LEGO')
plt.xticks(fontsize=8)
plt.yticks(fontsize=11)
plt.ylabel('Wartość koszyka (w EUR)', fontsize=11)
plt.xlabel('Data', fontsize=11)
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()